<a href="https://colab.research.google.com/github/Kobeieii/Project/blob/main/Data_Pipeline(Apache%20Airflow).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-airflow
!pip install pymysql

In [ ]:
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago
from datetime import timedelta

import pymysql.cursors
import pandas as pd
import requests

ตรงนี้เป็นข้อมูลสำคัญ ขออนุญาตไม่บอกนะคะ

In [ ]:
class Config:
    MYSQL_HOST = ''
    MYSQL_PORT = 3306
    MYSQL_USER = ''
    MYSQL_PASSWORD = ''
    MYSQL_DB = ''
    MYSQL_CHARSET = ''

#Get data from database

*   ดึงข้อมูล retail จาก database
*   จากนั้นนำข้อมูลมาใส่ Dataframe
*   เนื่องจาก InvoiceDate column ที่ได้มามีทั้ง ปี/เดือน/วัน เวลา เราไม่ได้ต้องการ timestamp จึงสร้างคอลลัมน์ ใหม่ไว้เก็บ Timestamp(เผื่ออนาคตได้ใช้) แล้ว format คอลัมน์เดิมให้เหลือแค่ ปี/เดือน/วัน

In [ ]:
def get_data_from_db():

    # Connect to the database
    connection = pymysql.connect(host=Config.MYSQL_HOST,
                                port=Config.MYSQL_PORT,
                                user=Config.MYSQL_USER,
                                password=Config.MYSQL_PASSWORD,
                                db=Config.MYSQL_DB,
                                charset=Config.MYSQL_CHARSET,
                                cursorclass=pymysql.cursors.DictCursor)


    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT * from online_retail"
        cursor.execute(sql)
        result_retail = cursor.fetchall()

    
    retail = pd.DataFrame(result_retail)
    retail['InvoiceTimestamp'] = pd.to_datetime(retail['InvoiceDate'])
    retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate']).dt.date
    retail.to_csv("/home/airflow/gcs/data/retail_from_db.csv", index=False)

#Get data from API

*   ดึงข้อมูลเรทเงินของแต่ละวันจาก api
*   นำข้อมูลมาใส่ dataframe เปลี่ยนชื่อคอลัมน์ที่เป็น index ตั้งชื่อใหม่ว่า Date




In [ ]:
def get_data_from_api():
    url = "https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all"
    response = requests.get(url)
    result_conversion_rate = response.json()

    conversion_rate = pd.DataFrame.from_dict(result_conversion_rate)
    conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})

    conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
    conversion_rate.to_csv("/home/airflow/gcs/data/conversion_rate_from_api.csv", index=False)

#Convert to Thai bath



*   นำข้อมูล retail กับข้อมูล currencry rate มา merge กันแบบ left join โดยให้ข้อมูล retail เป็นหลัก
*   เพิ่มคอลัมน์ 'THBPrice' ที่เกิดจากแปลงค่าเงินโดยการนำคอลัมน์ UnitPrice * Rate



In [ ]:
def convert_to_thb():
    retail = pd.read_csv("/home/airflow/gcs/data/retail_from_db.csv")
    conversion_rate = pd.read_csv("/home/airflow/gcs/data/conversion_rate_from_api.csv")
    
    final_df = retail.merge(conversion_rate, how="left", left_on="InvoiceDate", right_on="date")

    final_df['THBPrice'] = final_df.apply(lambda x: x['UnitPrice'] * x['Rate'], axis=1)
    final_df.to_csv("/home/airflow/gcs/data/result.csv", index=False)

#Data pipeline

ตั้ง Default Argument ให้ apache airflow

In [ ]:
default_args = {
    'owner': 'User_ploy',
    'depends_on_past': False,
    'start_date': days_ago(1),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    'schedule_interval': '@once',
}

Create DAGs (Directed Acyclic Graphs) 

In [ ]:
dag = DAG(
    'online_retail',
    default_args=default_args,
    description='Pipeline for ETL online_retail data',
    schedule_interval=timedelta(days=1),
)

In [ ]:
# db_ingest
t1 = PythonOperator(
    task_id='db_ingest',
    python_callable=get_data_from_db,
    dag=dag,
)

# api_call
t2 = PythonOperator(
    task_id='api_call',
    python_callable=get_data_from_api,
    dag=dag,
)

# convert_currency
t3 = PythonOperator(
    task_id='convert_currency',
    python_callable=convert_to_thb,
    dag=dag,
)

# load to BigQuery
t4 = BashOperator(
    task_id='bq_load',
    bash_command='bq load --source_format=CSV --autodetect \
            [DATASET_ID].[TABLE_NAME] \
            gs://[GCS_BUCKET_NAME]/data/result.csv',
    dag=dag,
)

# Dependencies

[t1, t2] >> t3 >> t4

<Task(BashOperator): bq_load>